<a href="https://colab.research.google.com/github/mrneerajnegi/Data-Science-Projects/blob/auto_ml/Flight%20Fare%20Prediction%20Using%20AutoSKLearn/Flight_Fare_Prediction_Using_Auto_SK_Learn_(Auto_ML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import re

In [67]:
#pip install auto-sklearn

In [68]:
flight_train=pd.read_csv("./flight data.csv")
flight_test=pd.read_csv("./flight test.csv")


In [69]:
flight_train=flight_train.dropna()

In [70]:
df=flight_train.append(flight_test)

In [71]:
df["Airline"].value_counts()


Jet Airways                          4746
IndiGo                               2564
Air India                            2191
Multiple carriers                    1543
SpiceJet                             1026
Vistara                               608
Air Asia                              405
GoAir                                 240
Multiple carriers Premium economy      16
Jet Airways Business                    8
Vistara Premium economy                 5
Trujet                                  1
Name: Airline, dtype: int64

In [72]:
def labelEncoder(columnName):
  labelEncoder=LabelEncoder()
  newData=labelEncoder.fit_transform(df[columnName])
  return newData

In [73]:
df["Airline"]=labelEncoder("Airline")
df["Source"]=labelEncoder("Source")
df["Destination"]=labelEncoder("Destination")

In [74]:
df=df.drop("Date_of_Journey",axis=1)

In [75]:
def getHour(value):
  return int(re.findall('\d+',value)[0])

def getMin(value):
  return int(re.findall('\d+',value)[1])

df["dep_hour"]=df["Dep_Time"].apply(getHour)
df["dep_min"]=df["Dep_Time"].apply(getMin)
df["arr_hour"]=df["Arrival_Time"].apply(getHour)
df["arr_min"]=df["Arrival_Time"].apply(getMin)

In [76]:
df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,dep_hour,dep_min,arr_hour,arr_min
0,3,0,5,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,22,20,1,10
1,1,3,0,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,5,50,13,15
2,4,2,1,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,9,25,4,25
3,3,3,0,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,18,5,23,30
4,3,0,5,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,16,50,21,35


In [77]:
def getDuration(value):
   time=re.findall('\d+',value)
   return float(time[0]+"."+time[1] if len(time)>1 else 0 )

In [78]:
df["Duration"]=df["Duration"].apply(getDuration)

In [79]:
def getStops(value):
  if value=="non-stop":
      return 0
  else:
   return int(value.split(" ")[0])

In [80]:
df["Total_Stops"]=df["Total_Stops"].apply(getStops)

In [81]:
df

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,dep_hour,dep_min,arr_hour,arr_min
0,3,0,5,BLR → DEL,22:20,01:10 22 Mar,2.50,0,No info,3897.0,22,20,1,10
1,1,3,0,CCU → IXR → BBI → BLR,05:50,13:15,7.25,2,No info,7662.0,5,50,13,15
2,4,2,1,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,0.00,2,No info,13882.0,9,25,4,25
3,3,3,0,CCU → NAG → BLR,18:05,23:30,5.25,1,No info,6218.0,18,5,23,30
4,3,0,5,BLR → NAG → DEL,16:50,21:35,4.45,1,No info,13302.0,16,50,21,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,1,3,0,CCU → DEL → BLR,20:30,20:25 07 Jun,23.55,1,No info,NaN,20,30,20,25
2667,3,3,0,CCU → BLR,14:20,16:55,2.35,0,No info,NaN,14,20,16,55
2668,4,2,1,DEL → BOM → COK,21:50,04:25 07 Mar,6.35,1,No info,NaN,21,50,4,25
2669,1,2,1,DEL → BOM → COK,04:00,19:15,15.15,1,No info,NaN,4,0,19,15


In [82]:
df["Additional_Info"].value_counts()

No info                         10492
In-flight meal not included      2426
No check-in baggage included      396
1 Long layover                     20
Change airports                     8
Business class                      5
No Info                             3
1 Short layover                     1
Red-eye flight                      1
2 Long layover                      1
Name: Additional_Info, dtype: int64

In [83]:
df["Additional_Info"]=labelEncoder("Additional_Info")

In [84]:
df["Additional_Info"]

0       8
1       8
2       8
3       8
4       8
       ..
2666    8
2667    8
2668    8
2669    8
2670    8
Name: Additional_Info, Length: 13353, dtype: int64

In [85]:
df["Route1"]=df["Route"].apply(lambda x : x.split(" → ")[0])
df["Route2"]=df["Route"].apply(lambda x : x.split(" → ")[1])
df["Route3"]=df["Route"].apply(lambda x : x.split(" → ")[2] if len(x.split(" → "))>2 else "No")
df["Route4"]=df["Route"].apply(lambda x : x.split(" → ")[3] if len(x.split(" → "))>3 else "No")

In [86]:
df.drop(["Arrival_Time","Route","Dep_Time","Duration"],inplace=True,axis=1)

In [87]:
df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,dep_hour,dep_min,arr_hour,arr_min,Route1,Route2,Route3,Route4
0,3,0,5,0,8,3897.0,22,20,1,10,BLR,DEL,No,No
1,1,3,0,2,8,7662.0,5,50,13,15,CCU,IXR,BBI,BLR
2,4,2,1,2,8,13882.0,9,25,4,25,DEL,LKO,BOM,COK
3,3,3,0,1,8,6218.0,18,5,23,30,CCU,NAG,BLR,No
4,3,0,5,1,8,13302.0,16,50,21,35,BLR,NAG,DEL,No


In [88]:
df["Route1"]=labelEncoder("Route1")
df["Route2"]=labelEncoder("Route2")
df["Route3"]=labelEncoder("Route3")
df["Route4"]=labelEncoder("Route4")

In [89]:
df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,dep_hour,dep_min,arr_hour,arr_min,Route1,Route2,Route3,Route4
0,3,0,5,0,8,3897.0,22,20,1,10,0,10,21,10
1,1,3,0,2,8,7662.0,5,50,13,15,2,20,1,3
2,4,2,1,2,8,13882.0,9,25,4,25,3,27,4,5
3,3,3,0,1,8,6218.0,18,5,23,30,2,29,3,10
4,3,0,5,1,8,13302.0,16,50,21,35,0,29,7,10


In [90]:
import autosklearn.regression as reg
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics



In [91]:
testdf=df[df["Price"].isna()]
testdf.drop(["Price"],axis=1,inplace=True)
traindf=df[df["Price"].notnull()]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [92]:
traindf["Price"]

0         3897.0
1         7662.0
2        13882.0
3         6218.0
4        13302.0
          ...   
10678     4107.0
10679     4145.0
10680     7229.0
10681    12648.0
10682    11753.0
Name: Price, Length: 10682, dtype: float64

In [93]:
df

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,dep_hour,dep_min,arr_hour,arr_min,Route1,Route2,Route3,Route4
0,3,0,5,0,8,3897.0,22,20,1,10,0,10,21,10
1,1,3,0,2,8,7662.0,5,50,13,15,2,20,1,3
2,4,2,1,2,8,13882.0,9,25,4,25,3,27,4,5
3,3,3,0,1,8,6218.0,18,5,23,30,2,29,3,10
4,3,0,5,1,8,13302.0,16,50,21,35,0,29,7,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,1,3,0,1,8,NaN,20,30,20,25,2,10,3,10
2667,3,3,0,0,8,NaN,14,20,16,55,2,5,21,10
2668,4,2,1,1,8,NaN,21,50,4,25,3,6,6,10
2669,1,2,1,1,8,NaN,4,0,19,15,3,6,6,10


In [94]:
train_x=traindf.drop(["Price"],axis=1)
train_y=traindf["Price"]

In [95]:
automl=reg.AutoSklearnRegressor(time_left_for_this_task=120,per_run_time_limit=30)

In [96]:
automl.fit(train_x,train_y)

AutoSklearnRegressor(per_run_time_limit=30, time_left_for_this_task=120)

In [97]:
automl.show_models()

{3: {'cost': 0.2428909152074049,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x7f9c03752650>,
  'ensemble_weight': 0.64,
  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice at 0x7f9c037daf90>,
  'model_id': 3,
  'rank': 1,
  'regressor': <autosklearn.pipeline.components.regression.RegressorChoice at 0x7f9c037da890>,
  'sklearn_regressor': HistGradientBoostingRegressor(l2_regularization=5.997418027353535e-10,
                                learning_rate=0.12286466971783992, max_iter=512,
                                max_leaf_nodes=26, min_samples_leaf=8,
                                n_iter_no_change=0, random_state=1,
                                validation_fraction=None, warm_start=True)},
 6: {'cost': 0.311611859325031,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x7f9c0370fb90>,
  'ensemble_weight': 0.16,
  

In [98]:
ypred= automl.predict(testdf)

In [102]:
ypred

array([15100.29345703,  4959.38140869, 10778.11975098, ...,
       15121.04638672, 10134.66577148,  8809.28796387])

In [100]:
train_x.columns

Index(['Airline', 'Source', 'Destination', 'Total_Stops', 'Additional_Info',
       'dep_hour', 'dep_min', 'arr_hour', 'arr_min', 'Route1', 'Route2',
       'Route3', 'Route4'],
      dtype='object')

In [101]:
testdf.columns

Index(['Airline', 'Source', 'Destination', 'Total_Stops', 'Additional_Info',
       'dep_hour', 'dep_min', 'arr_hour', 'arr_min', 'Route1', 'Route2',
       'Route3', 'Route4'],
      dtype='object')